### Import and check data

In [ ]:
%run 'Load_and_Preprocessing.ipynb'

In [ ]:
df_order_products = df_order_products
df_orders= df_orders

### Merge dataframes for analytics
df_orders: orders with tip <br>
df_order_products: ordered products with informations of the order, department and aisle.

In [ ]:

import pandas as pd
import plotly.express as px

# Angenommen, df ist der DataFrame, der die relevanten Daten enthält
# Zuerst sortieren wir nach user_id und order_id (damit wir die Reihenfolge der Bestellungen haben)
df = df_orders.sort_values(by=["user_id", "order_id"])

# Erstellen einer neuen Spalte, die das Trinkgeldverhalten der vorherigen Bestellung anzeigt
#df["previous_tip"] = df.groupby("user_id")["tip"].shift(1)

df["previous_tip"] = df["tip"].shift(1)


# Filtern der Zeilen, bei denen es eine vorherige Bestellung gibt
df_filtered = df[df["previous_tip"].notna()]

# Untersuchen, ob es einen Zusammenhang zwischen dem Trinkgeldverhalten der vorherigen Bestellung und der aktuellen Bestellung gibt
# Wir erstellen einen einfachen Plot, der die Verteilung der Trinkgeldverhalten für aktuelle und vorherige Bestellungen anzeigt

fig = px.histogram(df_filtered, x="previous_tip", color="tip",
                   labels={"previous_tip": "Trinkgeld vorherige Bestellung",
                           "tip": "Trinkgeld aktuelle Bestellung"},
                   title="Zusammenhang zwischen Trinkgeld bei vorheriger und aktueller Bestellung",
                   category_orders={"tip": [True, False], "previous_tip": [True, False]})

fig.show()


In [ ]:
df

### 3 Aufgabenstellungen
## 3.1 Explorative Datenanalyse für Marketing und Disposition

Als erstes wird eine explorative Analyse des Datensatzes für die Marketing- und Dispositionsabteilung
erwartet. Ihre Aufgabe ist eine zielgruppengerechte Aufbereitung zweier „Data Stories“ (je Abteilung
eine) durch die die Stakeholder in der Lage sein sollen, das Geschäftsfeld besser zu durchdringen und
Rückschlüsse für die eigenen betrieblichen Handlungsbereiche aus den Daten zu ziehen. Die
Stakeholder erwarten hierfür im Jupyter-Notebook einen Bereich „EDA für Marketing und
Disposition“.

Für die Analyse sind zunächst einige Vorverarbeitungsschritte nötig. Nutzen Sie Ihre Kenntnisse in
Python (bzw. den besprochenen Bibliotheken), um die Daten grundsätzlich zu verstehen, zu
kombinieren und ggf. anzureichern.
Die Marketing-Abteilung erhofft sich Erkenntnisse, die zur Optimierung des Produktkatalogs und der
Produktplatzierung auf der statischen Website verwendet werden können. Wenn viele Kunden bspw.
„Flour“ und „Eggs“ zusammen kaufen, könnten beide Produkte nebeneinander auf der Website platziert
werden. Der Zusammenhang muss nicht zwingend wie im Beispiel auf Produktebene hergestellt
werden, sondern kann auch auf der Ebene Kategorie („department“) oder Gang („aisle“) erfolgen.
Die Disposition erwartet von Ihrem Team Erkenntnisse über Unterschiede im Kundenverhalten
bezüglich des Trinkgelds. So soll vermieden werden, dass manche Mitarbeiter des Unternehmens
regelmäßig mehr Trinkgeld erhalten als andere (bspw. durch eine geänderte Zuordnung der Kunden zu
den Kurieren).

In den jeweiligen Analysen sind Sie sehr frei. Nutzen Sie Ihre Fähigkeiten zur tabellarischen und/oder
grafischen Aufbereitung der Daten sowie die statistischen Methoden, die Sie im Laufe des Studiums
erlernt haben. Strukturieren Sie das Notebook nachvollziehbar und bereiten Sie die Erkenntnisse
zielgruppengerecht auf. Belegen Sie dabei Ihre Schlussfolgerungen anhand der Daten in
angemessenem Detailgrad. Die Kenntnis gängiger statistischer Kennzahlen und ihrer Interpretation
können Sie als bekannt voraussetzen.2

## 3.2 Konkrete Fragen der Stakeholder
Die Stakeholder haben folgende konkrete Aufgaben bzw. Fragen für Ihren Beratungsauftrag formuliert:

Frage 1: Hängt die Wahrscheinlichkeit, dass bei einer Bestellung Trinkgeld gegeben wird, vom
Trinkgeldverhalten bei früheren Bestellungen desselben Bestellers ab?

a)   Gibt es einen Zusammenhang bezüglich der vorhergehenden Bestellung?

b)   Gibt es einen Zusammenhang bezüglich der vor-vorhergehenden Bestellung?

c) Liefert das Trinkgeldverhalten der vor-vorhergehende Bestellung Informationen auch über das
hinaus, was bereits aus der vorhergehenden Bestellung abgelesen werden kann?

# Vorgehensweise für die drei Unterfragen

## a) Gibt es einen Zusammenhang bezüglich der vorhergehenden Bestellung?

	1.	Hypothesen:
	•	H0: Das Trinkgeldverhalten einer Bestellung hängt nicht vom Trinkgeldverhalten der vorhergehenden Bestellung ab.
	•	H1: Das Trinkgeldverhalten einer Bestellung hängt signifikant vom Trinkgeldverhalten der vorhergehenden Bestellung ab.
	2.	Analyse:
	•	Berechne für jede Bestellung, ob bei der vorhergehenden Bestellung Trinkgeld gegeben wurde (previous_tip).
	•	Erstelle eine Kreuztabelle, um die Häufigkeit der Kombinationen von tip und previous_tip darzustellen.
	•	Führe einen Chi-Quadrat-Test durch, um die Signifikanz des Zusammenhangs zu testen.
	3.	Ergebnisse:
	•	Präsentiere die Kreuztabelle und den p-Wert des Chi-Quadrat-Tests.
	•	Interpretiere den p-Wert: Ein p-Wert < 0,05 weist darauf hin, dass ein signifikanter Zusammenhang besteht.

Beispiel:
“Die Analyse zeigt, dass ein signifikanter Zusammenhang zwischen dem Trinkgeldverhalten der aktuellen Bestellung und der vorhergehenden Bestellung besteht (p-Wert = 0,03). Kunden, die bei der vorhergehenden Bestellung Trinkgeld gegeben haben, tendieren dazu, auch bei der aktuellen Bestellung Trinkgeld zu geben.”

In [ ]:
from scipy.stats import chi2_contingency

# Schritt 1: Daten vorbereiten
# Angenommen, df_orders enthält die Spalten 'user_id', 'order_number', und 'tip'
df_orders = df_orders.sort_values(by=['user_id', 'order_number'])  # Sortieren nach Benutzer und Bestellnummer

# Schritt 2: Trinkgeld bei der vorhergehenden Bestellung berechnen
df_orders['previous_tip'] = df_orders.groupby('user_id')['tip'].shift(1)  # Vorhergehendes Trinkgeld
df_orders['previous_tip'] = df_orders['previous_tip'].fillna(False).astype(bool)  # NaN als False behandeln

# Schritt 3: Kreuztabelle erstellen
contingency_table = pd.crosstab(df_orders['tip'], df_orders['previous_tip'])

# Schritt 4: Chi-Quadrat-Test durchführen
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Ergebnisse anzeigen
print("Kreuztabelle der Häufigkeiten:")
print(contingency_table)
print("\nChi-Quadrat-Test:")
print(f"Chi2-Wert: {chi2:.2f}")
print(f"p-Wert: {p:.4f}")
print(f"Freiheitsgrade: {dof}")

# Interpretation
if p < 0.05:
    print("\nErgebnis: Es besteht ein signifikanter Zusammenhang zwischen dem Trinkgeldverhalten der aktuellen und der vorhergehenden Bestellung.")
else:
    print("\nErgebnis: Es besteht kein signifikanter Zusammenhang zwischen dem Trinkgeldverhalten der aktuellen und der vorhergehenden Bestellung.")

In [ ]:
df